In [1]:
# 1 – Install main dependencies
!pip install -q --upgrade "numpy<2.0"  # ۱.۲۶.۴
!pip install -q torch==2.2.0+cu118 torchtext==0.17.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q torch_geometric torch_sparse torch_scatter torch_cluster torch_spline_conv \
              -f https://data.pyg.org/whl/torch-2.2.0+cu118.html


In [2]:
# 2 – Connect Google Drive and set paths
from google.colab import drive
drive.mount('/content/drive')

GRAPH_PATH = "/content/drive/MyDrive/ml100k_graphs/ml100k_step3_full_graph.pt"
MODEL_DIR  = "/content/drive/MyDrive/ml100k_models"

import os, torch, numpy as np, random
os.makedirs(MODEL_DIR, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Mounted at /content/drive
Using device: cuda


In [3]:
# 3 – Load graph, add self-loops, and advanced metapaths
from torch_geometric.transforms import AddSelfLoops, AddMetaPaths
from torch_geometric.data import HeteroData

data: HeteroData = torch.load(GRAPH_PATH)

# Self-loop on all relations 
data = AddSelfLoops(attr=None)(data)

# Metapaths
metapaths = [
    [("user","Rank","item"), ("item","BeRanked","user")],            # U-I-U 
    [("item","BeRanked","user"), ("user","Rank","item")],            # I-U-I 

    [("user","UserSim","user"), ("user","Rank","item")],             # m0
    [("user","Rank","item"), ("item","ItemSim","item")],             # m1

    [("user","memberOf","user_cluster"),
     ("user_cluster","contains","user"),
     ("user","Rank","item")],                                        # m2

    [("user","Rank","item"),
     ("item","memberOf","item_cluster"),
     ("item_cluster","contains","item")],                            # m3

    [("user","UserSim","user"),
     ("user","Rank","item"),
     ("item","ItemSim","item")],                                     # m4

    [("user","memberOf","user_cluster"),
     ("user_cluster","contains","user"),
     ("user","Rank","item"),
     ("item","ItemSim","item")]                                      # m5
]

data = AddMetaPaths(metapaths, max_sample=300)(data)
print("Edge types after AddMetaPaths →", data.edge_types)
data = data.to(device)


/usr/local/lib/python3.11/dist-packages/torch_geometric/edge_index.py:863: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


Edge types after AddMetaPaths → [('user', 'Rank', 'item'), ('item', 'BeRanked', 'user'), ('user', 'UserSim', 'user'), ('item', 'ItemSim', 'item'), ('user', 'memberOf', 'user_cluster'), ('user_cluster', 'contains', 'user'), ('item', 'memberOf', 'item_cluster'), ('item_cluster', 'contains', 'item'), ('user', 'metapath_0', 'user'), ('item', 'metapath_1', 'item'), ('user', 'metapath_2', 'item'), ('user', 'metapath_3', 'item'), ('user', 'metapath_4', 'item'), ('user', 'metapath_5', 'item'), ('user', 'metapath_6', 'item'), ('user', 'metapath_7', 'item')]


In [4]:
# 4 – Prepare train / test indices for the Rank relation
rank_rel = ("user","Rank","item")

ei          = data[rank_rel].edge_index
ratings_all = data[rank_rel].rating.to(device)

train_mask  = data[rank_rel].train_mask
test_mask   = data[rank_rel].test_mask

train_src, train_dst = ei[:, train_mask]
test_src , test_dst  = ei[:, test_mask]

y_train = ratings_all[train_mask]
y_test  = ratings_all[test_mask]

print(f"Train edges: {y_train.numel():,}  |  Test edges: {y_test.numel():,}")


Train edges: 95,000  |  Test edges: 5,000


In [5]:
# 5 – Three-layer HAN architecture + custom edge-dropout
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HANConv
from torch_geometric.utils import dropout_edge        

class HANRecommender(nn.Module):
    def __init__(self, metadata, in_dims, hidden=128, out=128,
                 heads=(8, 4, 1), dropout=0.4, edge_p=0.2):
        super().__init__()
        self.edge_p = edge_p         # Edge dropout probability

        self.h1 = HANConv(in_dims, hidden, metadata,
                          heads=heads[0], dropout=dropout)
        self.h2 = HANConv(-1, hidden, metadata,
                          heads=heads[1], dropout=dropout)
        self.h3 = HANConv(-1, out,    metadata,
                          heads=heads[2], dropout=dropout)

        self.link = nn.Sequential(
            nn.Linear(2*out, out),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(out, 1)
        )

    def _edge_dropout(self, edge_index_dict):
        """Apply edge dropout to each relation."""
        if not self.training or self.edge_p == 0:
            return edge_index_dict
        out = {}
        for rel, ei in edge_index_dict.items():
            ei_dropped, _ = dropout_edge(ei, p=self.edge_p, force_undirected=False)
            out[rel] = ei_dropped
        return out

    def forward(self, x_dict, edge_index_dict, src, dst):
        edge_index_dict = self._edge_dropout(edge_index_dict)

        x_dict = self.h1(x_dict, edge_index_dict)
        x_dict = {k: F.relu(v) for k, v in x_dict.items()}

        x_dict = self.h2(x_dict, edge_index_dict)
        x_dict = {k: F.relu(v) for k, v in x_dict.items()}

        x_dict = self.h3(x_dict, edge_index_dict)

        h_u, h_v = x_dict["user"][src], x_dict["item"][dst]
        return self.link(torch.cat([h_u, h_v], dim=-1)).squeeze()

# Input dimensions for each node type
in_dims = {nt: data[nt].num_features for nt in data.node_types}

model = HANRecommender(data.metadata(), in_dims).to(device)
opt   = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-5)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=50)
print("Model ready ✔")


Model ready ✔ (edge-dropout فعال)


In [9]:
# Training with Early-Stopping and validation
from math import sqrt
import torch.nn.functional as F

# Split Train into Train/Val 
train_mask = data[rank_rel].train_mask.bool()          # (100 000,)
train_idx  = torch.nonzero(train_mask, as_tuple=False).view(-1)
n_val      = int(0.10 * train_idx.numel())             # 10٪

# Randomly select validation indices
val_sel    = train_idx[torch.randperm(train_idx.numel(), generator=torch.Generator().manual_seed(42))[:n_val]]
val_mask   = torch.zeros_like(train_mask)
val_mask[val_sel] = True

# Final masks
train_sub = train_mask & ~val_mask
val_sub   = val_mask

train_src_, train_dst_ = ei[:, train_sub]
val_src_,   val_dst_   = ei[:, val_sub]

y_train = ratings_all[train_sub]
y_val   = ratings_all[val_sub]

# Training loop
best_rmse, patience, PATIENCE = 1e9, 0, 8   # early-stop

for epoch in range(1, 101):
    model.train()
    opt.zero_grad()

    pred = model(data.x_dict, data.edge_index_dict, train_src_, train_dst_)
    loss = F.mse_loss(pred, y_train)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 3.0)
    opt.step()
    sched.step()

    # Validation
    model.eval()
    with torch.no_grad():
        p_val   = model(data.x_dict, data.edge_index_dict, val_src_, val_dst_)
        rmse_val = sqrt(F.mse_loss(p_val, y_val).item())

    # ─ Early-Stopping
    if rmse_val < best_rmse - 1e-3:
        best_rmse, patience = rmse_val, 0
        torch.save(model.state_dict(), f"{MODEL_DIR}/best_han.pt")
    else:
        patience += 1

    if epoch % 10 == 0 or epoch == 1:
        print(f"E{epoch:03d} | trainloss {loss.item():.3f} | val RMSE {rmse_val:.4f}")

    if patience >= PATIENCE:
        print("Early stop ◀")
        break


E001 | trainloss 1.438 | val RMSE 1.2032
Early stop ◀


In [7]:
# Final evaluation on test
best = HANRecommender(data.metadata(), in_dims).to(device)
best.load_state_dict(torch.load(f"{MODEL_DIR}/best_han.pt"))
best.eval()

with torch.no_grad():
    p_test = best(data.x_dict, data.edge_index_dict, test_src, test_dst)
    rmse   = sqrt(F.mse_loss(p_test, y_test).item())
    mae    = torch.mean(torch.abs(p_test - y_test)).item()

print(f"🔚  Final  →  RMSE = {rmse:.4f}   |   MAE = {mae:.4f}")


🔚  Final  →  RMSE = 1.1547   |   MAE = 0.9406
